In [1]:
import numpy as np
import pyscf
import matplotlib.pyplot as plt
import pyscf.lo


In [2]:
mol = pyscf.gto.M(atom="H 0. 0. 0.; H 0. 0. 1.4",
                  unit='B',
                  basis='minao')
mf = pyscf.scf.RHF(mol)
mf.kernel()

converged SCF energy = -1.09056621169749


-1.0905662116974866

In [3]:
def get_iao(mol, mf, mo_coeff, ref_bas = 'minao'):
  a = pyscf.lo.iao.iao(mol, mo_coeff, minao=ref_bas) 
  a = pyscf.lo.vec_lowdin(a, mf.get_ovlp())
  R_ao_lo = np.einsum('ji,jk->ik', a, mf.get_ovlp())
  return a, R_ao_lo

a, R_ao_lo = get_iao(mol, mf, mf.mo_coeff[:,:2])
R_mo_lo = np.einsum('ji,jk,kl->il',a,mf.get_ovlp(),mf.mo_coeff[:,:2])

In [4]:
# Obtain the 1e TB model Hamiltonian in LO
ham_tb_lo = np.einsum('i,ai,bi->ab', mf.mo_energy, R_mo_lo, R_mo_lo)
print('TB Ham:\n', ham_tb_lo)

# check that the TB model gives the same mo energies
eigvals, eigvecs = np.linalg.eigh(ham_tb_lo)
print('TB eigenvals:', eigvals)
print('HF mo energies:', mf.mo_energy)

TB Ham:
 [[-0.11020441 -0.50912646]
 [-0.50912646 -0.11020441]]
TB eigenvals: [-0.61933087  0.39892205]
HF mo energies: [-0.61933087  0.39892205]


In [5]:
# compute the 1e integrals
ham_1e_kin = pyscf.gto.getints("int1e_kin_sph", mol._atm, mol._bas, mol._env)
ham_1e_nuc = pyscf.gto.getints("int1e_nuc_sph", mol._atm, mol._bas, mol._env)

ham_1e = ham_1e_kin + ham_1e_nuc

# transform into MO basis
ham_1e_mo = np.einsum('kl,ki,lj', ham_1e, mf.mo_coeff, mf.mo_coeff)
print('direct 1e integrals in MO:\n', ham_1e_mo)


direct 1e integrals in MO:
 [[-1.18552106e+00 -1.11022302e-16]
 [-2.22044605e-16 -5.73440850e-01]]


In [6]:
# transform into the localized basis
ham_1e_lo = np.einsum('kl,ki,lj', ham_1e, a, a)
print('direct 1e integrals in LO:\n', ham_1e_lo)

direct 1e integrals in LO:
 [[-0.87948095 -0.3060401 ]
 [-0.3060401  -0.87948095]]


In [7]:
# compute the 2e integrals
int_2e_ao = pyscf.gto.getints("int2e_sph", mol._atm, mol._bas, mol._env)
int_2e_mo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff)

print('direct 2e integrals in MO:\n',int_2e_mo)

direct 2e integrals in MO:
 [[[[ 5.66190189e-01  1.38777878e-16]
   [ 0.00000000e+00  5.56277523e-01]]

  [[ 1.11022302e-16  1.40192148e-01]
   [ 1.40192148e-01  0.00000000e+00]]]


 [[[-5.55111512e-17  1.40192148e-01]
   [ 1.40192148e-01 -3.33066907e-16]]

  [[ 5.56277523e-01  0.00000000e+00]
   [-3.33066907e-16  5.85863852e-01]]]]


In [8]:
# transform to the localized orbital basis
int_2e_lo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, a, a, a, a)

print('direct 2e integrals in LO:\n',int_2e_lo)

direct 2e integrals in LO:
 [[[[ 0.70634442 -0.00491842]
   [-0.00491842  0.42596012]]

  [[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]]


 [[[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]

  [[ 0.42596012 -0.00491842]
   [-0.00491842  0.70634442]]]]


##### Next, we compare the 1b and 2b energy contributions from the ab-initio (RHF) with those obtained using exact diagonalization of the model Hamiltonian

* RHF: 1b and 2b contributions 

In [9]:
rdm1_lo = np.einsum('ij,ai,bj->ab',mf.make_rdm1(), R_ao_lo, R_ao_lo)
e_1b = np.einsum('ij,ij->',ham_1e_lo, rdm1_lo)

rdm2_mo = np.zeros([2,2,2,2])
rdm2_mo[0,0,0,0] = 2
rdm2_lo = np.einsum('ijkl,ai,bj,ck,dl->abcd',rdm2_mo, R_mo_lo, R_mo_lo, R_mo_lo, R_mo_lo)
e_2b = 0.5*np.einsum('ijkl,ijkl->',int_2e_lo, rdm2_lo)

print('HF energy contribution from 1b and 2b:', e_1b, e_2b)
print('HF total energy - E_nuc:', e_1b+e_2b)

HF energy contribution from 1b and 2b: -2.3710421147640943 0.5661901887808957
HF total energy - E_nuc: -1.8048519259831988


* Exact diagonalization:

In [10]:
from pyscf import fci

def solver(n,h1,h2):
  nelec=(int(n/2),int(n/2))
  e, fcivec = fci.direct_spin1.kernel(h1, h2, n, nelec, verbose=5)
  dm1, dm2 = fci.direct_spin1.make_rdm12(fcivec, n, nelec)
  return {'e':e, 'dm1':dm1, 'dm2':dm2}

def get_energy_components(n,h1,h2):
  data = solver(n,h1,h2)
  int_expectation = 0.5*np.einsum('ijkl,ijkl->', data['dm2'], h2)
  oneb_expectation = np.einsum('ij,ij->', data['dm1'], h1)

  return {'total_energy_model':data['e'], '2b_expectation':int_expectation, 
          '1b_expectation':oneb_expectation} 

In [11]:
get_energy_components(2,ham_1e_lo,int_2e_lo)

{'total_energy_model': -1.8204571546696,
 '2b_expectation': 0.5356025005062913,
 '1b_expectation': -2.3560596551758923}